<a href="https://colab.research.google.com/github/h4ck4l1/datasets/blob/main/credit_risk/credit_risk_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'''

'بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'

In [2]:
'''سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'''

'سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'

In [3]:
# @title
# from IPython.core.display import HTML
# HTML(r"""
# <style>
#     * {
#         font-family: Monaco;
#     }
# </style>
# """)

def increase_font():
  from IPython.display import Javascript
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      rule.style.fontSize = '18px'
      break
    }
  }
  '''))

get_ipython().events.register('pre_run_cell', increase_font)

# Imports and Processing

In [4]:
import os,sys,warnings,time,re,math,gc
from IPython.display import clear_output
if "google.colab" in sys.modules:
    !pip install wget
    !pip install --upgade "polars[all]"
    clear_output()

In [5]:
%xmode Minimal
warnings.filterwarnings("ignore")
from typing import Literal,Optional
from textwrap import wrap
from pathlib import Path
import wget
import numpy as np
import polars as pl
from zipfile import ZipFile
import polars.selectors as cs
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Monaco",size=15)))
pio.templates.default = "plotly_dark+mod"
from plotly.subplots import make_subplots

<IPython.core.display.Javascript object>

Exception reporting mode: Minimal


In [6]:
path_to_train = None
if "google.colab" in sys.modules:
    os.mkdir("Downloads")
    wget.download("https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/50160/7602123/compressed/parquet_files.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1709737137&Signature=jB075gQGOmyHUzTwGJUUuYJl9gwt8iFA%2F%2F3ez3UFCl0GgVeFWpQC92zrLl37MF%2F%2BLx7dMigTAhqsJ%2BkB5km8uY2vAsn4A84nBUUuPf74zEC2qnpivtFwhBbvjUeA2Uc0wNmtEda05eJnJEvtxb2tdyCxVgSouUQ1x0LG63bvUs5EtvFcvUl2OO7R4AvDJTXUh99ZhNnjwfcB7OeUJXHlDyfHd1Mb2mkSDDXgS6tLLPolyu%2F4TCbyQoON0DbKCsujnD5AjCzOO2P6kElwrjExR7sS8zGJ9VLni3%2BACZMrjA%2F1UDSzrNNkGZ4EoA1UFEdODqPLdpPzMkFlWK1bwKYmnA%3D%3D&response-content-disposition=attachment%3B+filename%3Dparquet_files.zip","Downloads/parquets.zip")
    wget.download("https://storage.googleapis.com/kagglesdsdata/competitions/50160/7602123/feature_definitions.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1709738267&Signature=klHf3kuFN5Vd2IbWvYpUrJ7RTwekDAW1J6eh%2Fn%2FwkEE3c2HYmNtPNA9ZOCCDg73kxf2h82D%2BnMuvec1462cmKDMSF%2FIhzvu7DZybxm%2FJYq18slT5X66sYaw4MbamD93wFjPclwHGw4oNEaOkyvftMF4egKNqH5s4leSAYOc%2B%2BWtSPToN7OO%2BmKxejgoXoAimWxMOin8ifv25WAhwEeG3aq6fJlJJazEZ2nKiSQouBm0CcSthJ1%2B%2BYzAAQXxIlaLKJGmPX2P8jb61z2dSkWlOBpFCpbhS0ocbLdPEhtA1xQFgWeprhcQ4G7aX2vs%2FJFI1OYQrSQhBO%2FCZJ0VsTwXljA%3D%3D&response-content-disposition=attachment%3B+filename%3Dfeature_definitions.csv","Downloads/feature_definitions.csv")
    with ZipFile("Downloads/parquets.zip","r") as f:
        f.extractall("Downloads/credit_risk/")

<IPython.core.display.Javascript object>

In [7]:
pl.Config.set_fmt_float("full")
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_float_precision(2)
pl.Config.set_tbl_rows(32)

<IPython.core.display.Javascript object>

polars.config.Config

In [8]:
path_to_train = Path("Downloads/credit_risk/train")
all_train_files = os.listdir(path_to_train)
print(len(all_train_files))
all_train_files.sort()
all_files_df = (
    pl.DataFrame({
        "index":range(len(all_train_files)),
        "filename":all_train_files
    },
    schema={"index":pl.Int8,"filename":pl.Utf8}
    )
)
all_files_df

<IPython.core.display.Javascript object>

32


index,filename
i8,str
0,"""train_applprev_1_0.parquet"""
1,"""train_applprev_1_1.parquet"""
2,"""train_applprev_2.parquet"""
3,"""train_base.parquet"""
4,"""train_credit_bureau_a_1_0.parquet"""
5,"""train_credit_bureau_a_1_1.parquet"""
6,"""train_credit_bureau_a_1_2.parquet"""
7,"""train_credit_bureau_a_1_3.parquet"""
8,"""train_credit_bureau_a_2_0.parquet"""


In [9]:
feature_columns = (
    pl.read_csv("Downloads/feature_definitions.csv")
    .select(
        pl.all().shrink_dtype()
    )
)

def rename_to_full(df):
    rename_dict = {}
    for _ in df.columns:
        des_str = feature_columns.filter(pl.col("Variable") == _)["Description"].item() if _ not in ["case_id","num_group1","num_group2"] else ""
        total_str = " ".join(_.split("_"))+" \n "+"("+des_str+")" if _ not in ["case_id","num_group1","num_group2"] else _
        rename_dict[_] = total_str
    return df.rename(rename_dict)

def preprocess(ind:int):
    return (
        pl.scan_parquet(path_to_train/all_train_files[ind])
        .drop(cs.contains('date'))
        .select(pl.all().shrink_dtype())
        .with_columns(cs.integer().cast(pl.Int32))
        .pipe(rename_to_full)
    )

def drop_null_columns(df,null_threshold:int=0):
    df = df.collect(streaming=True)
    return df[:,[not(s.null_count() > null_threshold) for s in df]]

def plot_nulls(df,show_legend=True):
    _ = df.collect(streaming=True)
    l = _.shape[0]
    new_col_names = {_:re.sub(r'\n','<br>',_) for _ in _.columns}
    return (
        px.bar(
            _
            .rename(new_col_names)
            .null_count()
            .transpose(include_header=True,column_names=["null_counts"])
            .with_columns(
                (pl.col("null_counts")/l).round(2).alias("percent")
            ),
            x="column",
            y="null_counts",
            color="column",
            hover_data=["column","percent"],
            hover_name="column",
            height=700,
            width=3000
        )
        .update_xaxes(showticklabels=False,title="All Columns")
        .update_yaxes(title="Null Counts")
        .update_layout(legend_tracegroupgap=10,showlegend=show_legend)
        .add_hline(y=l,line=dict(dash="dashdot"),annotation_text="Total rows")
        .show()
    )

print(gc.collect())
feature_columns.head()

<IPython.core.display.Javascript object>

664


Variable,Description
str,str
"""actualdpd_943P""","""Days Past Due (DPD) of previous contract (actual)."""
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""addres_district_368M""","""District of the person's address."""
"""addres_role_871L""","""Role of person's address."""
"""addres_zip_823M""","""Zip code of the address."""


## Base Files

In [10]:
all_files_df.filter(pl.col("filename").str.contains("base"))

<IPython.core.display.Javascript object>

index,filename
i8,str
3,"""train_base.parquet"""


In [11]:
train_base = (
    pl.scan_parquet(path_to_train/all_train_files[3])
    .select(pl.all().shrink_dtype())
    .with_columns(
        pl.col('case_id').cast(pl.Int32),
        pl.col("date_decision").str.to_date()
    )
)
gc.collect()
display(train_base.fetch().head())

<IPython.core.display.Javascript object>

case_id,date_decision,MONTH,WEEK_NUM,target
i32,date,i32,i8,i8
0,2019-01-03,201901,0,0
1,2019-01-03,201901,0,0
2,2019-01-04,201901,0,0
3,2019-01-03,201901,0,0
4,2019-01-04,201901,0,1


In [12]:
train_base.collect(streaming=True).describe()

<IPython.core.display.Javascript object>

describe,case_id,date_decision,MONTH,WEEK_NUM,target
str,f64,str,f64,f64,f64
"""count""",1526659.00,"""1526659""",1526659.00,1526659.00,1526659.00
"""null_count""",0.00,"""0""",0.00,0.00,0.00
"""mean""",1286076.57,null,201936.29,40.77,0.03
"""std""",718946.59,null,44.74,23.80,0.17
"""min""",0.00,"""2019-01-01""",201901.00,0.00,0.00
"""25%""",766198.00,null,201906.00,23.00,0.00
"""50%""",1357358.00,null,201910.00,40.00,0.00
"""75%""",1739023.00,null,202001.00,55.00,0.00
"""max""",2703454.00,"""2020-10-05""",202010.00,91.00,1.00


In [13]:
def missing_case_ids(df:pl.LazyFrame):
    return (
        df
        .with_context(
            train_base.select(pl.col("case_id").suffix("_base"))
        )
        .select(
            (~pl.col("case_id_base").is_in(pl.col("case_id"))).cast(pl.Int8).mean().alias("percentage of base case ids not present")
        )
        .collect(streaming=True)
    )


<IPython.core.display.Javascript object>

## Previous Files

In [14]:
all_files_df.filter(pl.col('filename').str.contains("prev"))

<IPython.core.display.Javascript object>

index,filename
i8,str
0,"""train_applprev_1_0.parquet"""
1,"""train_applprev_1_1.parquet"""
2,"""train_applprev_2.parquet"""


In [15]:
train_applprev_1 = preprocess(0)
train_applprev_2 = preprocess(1)
gc.collect()

<IPython.core.display.Javascript object>

0

In [16]:
display(train_applprev_1.fetch().head())
train_applprev_1.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,actualdpd 943P (Days Past Due (DPD) of previous contract (actual).),annuity 853A (Monthly annuity for previous applications.),byoccupationinc 3656910L (Applicant's income from previous applications.),cancelreason 3545846M (Application cancellation reason.),childnum 21L (Number of children in the previous application.),credacc actualbalance 314A (Actual balance on credit account.),credacc credlmt 575A (Credit card credit limit provided for previous applications.),credacc maxhisbal 375A (Maximal historical balance of previous credit account),credacc minhisbal 90A (Minimum historical balance of previous credit accounts.),credacc status 367L (Account status of previous credit applications.),credacc transactions 402L (Number of transactions made with the previous credit account of the applicant.),credamount 590A (Loan amount or card limit of previous applications.),credtype 587L (Credit type of previous application.),currdebt 94A (Previous application's current debt.),district 544M (District of the address used in the previous loan application.),downpmt 134A (Previous application downpayment amount.),dtlastpmt 581D (Date of last payment made by the applicant.),dtlastpmtallstes 3545839D (Date of the applicant's last payment.),education 1138M (Applicant's education level from their previous application.),employedfrom 700D (Employment start date from the previous application.),familystate 726L (Family State in previous application of applicant.),inittransactioncode 279L (Type of the initial transaction made in the previous application of the client.),isbidproduct 390L (Flag for determining if the product is a cross-sell in previous applications.),isdebitcard 527L (Previous application flag indicating if product being applied for is a debit card.),mainoccupationinc 437A (Client's main income amount in their previous application.),maxdpdtolerance 577P (Maximum DPD with tolerance (on previous application/s).),num_group1,outstandingdebt 522A (Amount of outstanding debt on the client's previous application.),pmtnum 8L (Number of payments made for the previous application.),postype 4733339M (Type of point of sale.),profession 152M (Profession of the client during their previous loan application.),rejectreason 755M (Reason for previous application rejection.),rejectreasonclient 4145042M (Reason for rejection of the client's previous application.),revolvingaccount 394A (Revolving account that was present in the applicant's previous application.),status 219L (Previous application status.),tenor 203L (Number of instalments in the previous application.)
i32,f32,f32,f32,str,f32,f32,f32,f32,f32,str,f32,f32,str,f32,str,f32,str,str,str,str,str,str,bool,bool,f32,f32,i32,f32,f32,str,str,str,str,f32,str,f32
2,0.00,640.20,null,"""a55475b1""",0.00,null,0.00,null,null,null,null,10000.00,"""CAL""",null,"""P136_108_173""",0.00,null,null,"""P97_36_170""","""2010-02-15""","""SINGLE""","""CASH""",false,null,8200.00,null,0,null,24.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",24.00
2,0.00,1682.40,null,"""a55475b1""",0.00,null,0.00,null,null,null,null,16000.00,"""CAL""",null,"""P136_108_173""",0.00,null,null,"""P97_36_170""","""2010-02-15""","""SINGLE""","""CASH""",false,null,8200.00,null,1,null,12.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",12.00
3,0.00,6140.00,null,"""P94_109_143""",null,null,0.00,null,null,null,null,59999.80,"""CAL""",null,"""P131_33_167""",0.00,null,null,"""P97_36_170""","""2018-05-15""","""MARRIED""","""CASH""",false,null,11000.00,null,0,null,12.00,"""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""",null,"""D""",12.00
4,0.00,2556.60,null,"""P24_27_36""",null,null,0.00,null,null,null,null,40000.00,"""CAL""",null,"""P194_82_174""",0.00,null,null,"""a55475b1""",null,null,"""CASH""",false,null,16000.00,null,0,null,24.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""T""",24.00
5,0.00,null,null,"""P85_114_140""",null,null,null,null,null,null,nul

2692


In [17]:
display(train_applprev_2.fetch().head())
train_applprev_2.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,actualdpd 943P (Days Past Due (DPD) of previous contract (actual).),annuity 853A (Monthly annuity for previous applications.),byoccupationinc 3656910L (Applicant's income from previous applications.),cancelreason 3545846M (Application cancellation reason.),childnum 21L (Number of children in the previous application.),credacc actualbalance 314A (Actual balance on credit account.),credacc credlmt 575A (Credit card credit limit provided for previous applications.),credacc maxhisbal 375A (Maximal historical balance of previous credit account),credacc minhisbal 90A (Minimum historical balance of previous credit accounts.),credacc status 367L (Account status of previous credit applications.),credacc transactions 402L (Number of transactions made with the previous credit account of the applicant.),credamount 590A (Loan amount or card limit of previous applications.),credtype 587L (Credit type of previous application.),currdebt 94A (Previous application's current debt.),district 544M (District of the address used in the previous loan application.),downpmt 134A (Previous application downpayment amount.),dtlastpmt 581D (Date of last payment made by the applicant.),dtlastpmtallstes 3545839D (Date of the applicant's last payment.),education 1138M (Applicant's education level from their previous application.),employedfrom 700D (Employment start date from the previous application.),familystate 726L (Family State in previous application of applicant.),inittransactioncode 279L (Type of the initial transaction made in the previous application of the client.),isbidproduct 390L (Flag for determining if the product is a cross-sell in previous applications.),isdebitcard 527L (Previous application flag indicating if product being applied for is a debit card.),mainoccupationinc 437A (Client's main income amount in their previous application.),maxdpdtolerance 577P (Maximum DPD with tolerance (on previous application/s).),num_group1,outstandingdebt 522A (Amount of outstanding debt on the client's previous application.),pmtnum 8L (Number of payments made for the previous application.),postype 4733339M (Type of point of sale.),profession 152M (Profession of the client during their previous loan application.),rejectreason 755M (Reason for previous application rejection.),rejectreasonclient 4145042M (Reason for rejection of the client's previous application.),revolvingaccount 394A (Revolving account that was present in the applicant's previous application.),status 219L (Previous application status.),tenor 203L (Number of instalments in the previous application.)
i32,f32,f32,f32,str,f32,f32,f32,f32,f32,str,f32,f32,str,f32,str,f32,str,str,str,str,str,str,bool,bool,f32,f32,i32,f32,f32,str,str,str,str,f32,str,f32
40704,0.00,7204.60,null,"""P94_109_143""",null,null,0.00,null,null,null,null,54000.00,"""CAL""",null,"""P147_6_101""",0.00,null,null,"""a55475b1""",null,null,"""CASH""",false,null,40000.00,null,0,null,12.00,"""P46_145_78""","""a55475b1""","""P198_131_9""","""P94_109_143""",null,"""D""",12.00
40734,0.00,3870.20,null,"""P94_109_143""",null,null,0.00,null,null,null,null,50000.00,"""CAL""",null,"""P111_148_100""",0.00,null,null,"""a55475b1""",null,null,"""CASH""",false,null,50000.00,null,0,null,18.00,"""P149_40_170""","""a55475b1""","""P45_84_106""","""P94_109_143""",null,"""D""",18.00
40737,0.00,2324.40,1.00,"""a55475b1""",0.00,null,0.00,null,null,null,null,30000.00,"""CAL""",0.00,"""a55475b1""",0.00,null,null,"""P97_36_170""","""2014-01-15""","""MARRIED""","""CASH""",false,null,16000.00,null,0,0.00,18.00,"""P46_145_78""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",18.00
40791,0.00,3044.40,null,"""P94_109_143""",null,null,0.00,null,null,null,null,44840.00,"""COL""",null,"""P19_11_176""",0.00,null,null,"""a55475b1""",null,null,"""POS""",false,null,30000.00,null,0,null,24.00,"""P177_117_192""","""a55475b1""","""P99_56_166""","""P94_109_143""",null,"""D""",24.00
40791,0.00,2320.80,1.00,"""a55475b1""",0.00,null,0.00,null,null,null,nul

2458


In [18]:
train_previous_application = pl.concat(
    [
        train_applprev_1.with_columns(pl.col("case_id").cast(pl.Int32)),
        train_applprev_2.with_columns(pl.col("case_id").cast(pl.Int32))
    ],
    how="vertical",
    rechunk=True
)
display(train_previous_application.fetch().head())
train_previous_application.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,actualdpd 943P (Days Past Due (DPD) of previous contract (actual).),annuity 853A (Monthly annuity for previous applications.),byoccupationinc 3656910L (Applicant's income from previous applications.),cancelreason 3545846M (Application cancellation reason.),childnum 21L (Number of children in the previous application.),credacc actualbalance 314A (Actual balance on credit account.),credacc credlmt 575A (Credit card credit limit provided for previous applications.),credacc maxhisbal 375A (Maximal historical balance of previous credit account),credacc minhisbal 90A (Minimum historical balance of previous credit accounts.),credacc status 367L (Account status of previous credit applications.),credacc transactions 402L (Number of transactions made with the previous credit account of the applicant.),credamount 590A (Loan amount or card limit of previous applications.),credtype 587L (Credit type of previous application.),currdebt 94A (Previous application's current debt.),district 544M (District of the address used in the previous loan application.),downpmt 134A (Previous application downpayment amount.),dtlastpmt 581D (Date of last payment made by the applicant.),dtlastpmtallstes 3545839D (Date of the applicant's last payment.),education 1138M (Applicant's education level from their previous application.),employedfrom 700D (Employment start date from the previous application.),familystate 726L (Family State in previous application of applicant.),inittransactioncode 279L (Type of the initial transaction made in the previous application of the client.),isbidproduct 390L (Flag for determining if the product is a cross-sell in previous applications.),isdebitcard 527L (Previous application flag indicating if product being applied for is a debit card.),mainoccupationinc 437A (Client's main income amount in their previous application.),maxdpdtolerance 577P (Maximum DPD with tolerance (on previous application/s).),num_group1,outstandingdebt 522A (Amount of outstanding debt on the client's previous application.),pmtnum 8L (Number of payments made for the previous application.),postype 4733339M (Type of point of sale.),profession 152M (Profession of the client during their previous loan application.),rejectreason 755M (Reason for previous application rejection.),rejectreasonclient 4145042M (Reason for rejection of the client's previous application.),revolvingaccount 394A (Revolving account that was present in the applicant's previous application.),status 219L (Previous application status.),tenor 203L (Number of instalments in the previous application.)
i32,f32,f32,f32,str,f32,f32,f32,f32,f32,str,f32,f32,str,f32,str,f32,str,str,str,str,str,str,bool,bool,f32,f32,i32,f32,f32,str,str,str,str,f32,str,f32
2,0.00,640.20,null,"""a55475b1""",0.00,null,0.00,null,null,null,null,10000.00,"""CAL""",null,"""P136_108_173""",0.00,null,null,"""P97_36_170""","""2010-02-15""","""SINGLE""","""CASH""",false,null,8200.00,null,0,null,24.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",24.00
2,0.00,1682.40,null,"""a55475b1""",0.00,null,0.00,null,null,null,null,16000.00,"""CAL""",null,"""P136_108_173""",0.00,null,null,"""P97_36_170""","""2010-02-15""","""SINGLE""","""CASH""",false,null,8200.00,null,1,null,12.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",12.00
3,0.00,6140.00,null,"""P94_109_143""",null,null,0.00,null,null,null,null,59999.80,"""CAL""",null,"""P131_33_167""",0.00,null,null,"""P97_36_170""","""2018-05-15""","""MARRIED""","""CASH""",false,null,11000.00,null,0,null,12.00,"""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""",null,"""D""",12.00
4,0.00,2556.60,null,"""P24_27_36""",null,null,0.00,null,null,null,null,40000.00,"""CAL""",null,"""P194_82_174""",0.00,null,null,"""a55475b1""",null,null,"""CASH""",false,null,16000.00,null,0,null,24.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""T""",24.00
5,0.00,null,null,"""P85_114_140""",null,null,null,null,null,null,nul

4095


In [19]:
train_previous_application_status = preprocess(2)
display(train_previous_application_status.fetch().head())
train_previous_application_status.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,cacccardblochreas 147M (Card blocking reason.),conts type 509L (Person contact type in previous application.),credacc cards status 52L (Card status of the previous credit account.),num_group1,num_group2
i32,str,str,str,i32,i32
2,null,"""EMPLOYMENT_PHONE""",null,1,1
2,null,"""EMPLOYMENT_PHONE""",null,0,1
2,null,"""PRIMARY_MOBILE""",null,0,0
2,null,"""PRIMARY_MOBILE""",null,1,0
3,null,"""PRIMARY_MOBILE""",null,0,1


3071


In [20]:
del train_applprev_1,train_applprev_2

<IPython.core.display.Javascript object>

## Static Files

In [21]:
all_files_df.filter(pl.col("filename").str.contains("static"))

<IPython.core.display.Javascript object>

index,filename
i8,str
26,"""train_static_0_0.parquet"""
27,"""train_static_0_1.parquet"""
28,"""train_static_cb_0.parquet"""


In [22]:
train_static_0 = preprocess(26)
display(train_static_0.fetch().head())
train_static_0.pipe(plot_nulls,show_legend=False)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,actualdpdtolerance 344P (DPD of client with tolerance.),amtinstpaidbefduel24m 4187115A (Number of instalments paid before due date in the last 24 months.),annuity 780A (Monthly annuity amount.),annuitynextmonth 57A (Next month's amount of annuity.),applicationcnt 361L (Number of applications associated with the same email address as the client.),applications30d 658L (Number of applications made by the client in the last 30 days.),applicationscnt 1086L (Number of applications associated with the same phone number.),applicationscnt 464L (Number of applications made in the last 30 days by other clients with the same employer as the applicant.),applicationscnt 629L (Number of applications with the same employer in the last 7 days.),applicationscnt 867L (Number of applications associated with the same mobile phone.),avgdbddpdlast24m 3658932P (Average days past or before due of payment during the last 24 months.),avgdbddpdlast3m 4187120P (Average days past or before due of payment during the last 3 months.),avgdbdtollast24m 4525197P (Average days of payment before due date within the last 24 months (with tolerance).),"avgdpdtolclosure24 3658938P (Average DPD (days past due) with tolerance within the past 24 months from the maximum closure date, assuming that the contract is finished. If the contract is ongoing, the calculation is based on the current date.)",avginstallast24m 3658937A (Average instalments paid by the client over the past 24 months.),avglnamtstart24m 4525187A (Average loan amount in the last 24 months.),avgmaxdpdlast9m 3716943P (Average Days Past Due (DPD) of the client in last 9 months.),avgoutstandbalancel6m 4187114A (Average outstanding balance of applicant for the last 6 months.),avgpmtlast12m 4525200A (Average of payments made by the client in the last 12 months.),bankacctype 710L (Type of applicant's bank account.),cardtype 51L (Type of credit card.),clientscnt12m 3712952L (Number of clients that have used the same mobile phone as the applicant in the past 12 months.),clientscnt3m 3712950L (Number of clients who have the same mobile phone number in the last 3 months.),clientscnt6m 3712949L (Total number of clients who have used the same mobile number in the last 6 months.),clientscnt 100L (Number of applications with matching employer's phone and client's.),clientscnt 1022L (Number of clients sharing the same mobile phone.),clientscnt 1071L (Number of applications where the alternative phone number matches that of the client.),clientscnt 1130L (Number of applications where client's phone number matches the alternative phone contact.),clientscnt 136L (Number of applications associated with same email address as client's email.),clientscnt 157L (Number of clients whose employer has the same phone number as the client.),clientscnt 257L (Number of clients that share an alternative phone number with the applicant.),clientscnt 304L (Number of clients with the same phone number.),clientscnt 360L (Number of clients that have the same alternative phone number and employer's phone number.),clientscnt 493L (Number of clients with matching phone numbers for both the employer and the client.),clientscnt 533L (Number of clients with same client's and alternative's phone number),clientscnt 887L (Number of clients sharing the same employer's phone number.),…,numinstpaidearlyest 4493214L (Number of instalments paid before the due date.),numinstpaidlastcontr 4325080L (Number of paid installments from the client's last contract.),numinstpaidlate1d 3546852L (Number of instalments paid more than 1 day past their due date.),numinstregularpaid 973L (Number of fully paid regular installments in the client's previous contracts.),numinstregularpaidest 4493210L (Number of fully paid regular installments on clients' previous contracts.),numinsttopaygr 769L (Number of unpaid instalments.),numinsttopaygrest 4493213L (Number of unpaid instalments.),numinstunpaidmax 3546851L (Maximum number of unpaid instalments.),numinstunpaidmaxest 4

7221


In [23]:
train_static_1 = preprocess(27)
display(train_static_1.fetch().head())
train_static_1.pipe(plot_nulls,show_legend=False)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,actualdpdtolerance 344P (DPD of client with tolerance.),amtinstpaidbefduel24m 4187115A (Number of instalments paid before due date in the last 24 months.),annuity 780A (Monthly annuity amount.),annuitynextmonth 57A (Next month's amount of annuity.),applicationcnt 361L (Number of applications associated with the same email address as the client.),applications30d 658L (Number of applications made by the client in the last 30 days.),applicationscnt 1086L (Number of applications associated with the same phone number.),applicationscnt 464L (Number of applications made in the last 30 days by other clients with the same employer as the applicant.),applicationscnt 629L (Number of applications with the same employer in the last 7 days.),applicationscnt 867L (Number of applications associated with the same mobile phone.),avgdbddpdlast24m 3658932P (Average days past or before due of payment during the last 24 months.),avgdbddpdlast3m 4187120P (Average days past or before due of payment during the last 3 months.),avgdbdtollast24m 4525197P (Average days of payment before due date within the last 24 months (with tolerance).),"avgdpdtolclosure24 3658938P (Average DPD (days past due) with tolerance within the past 24 months from the maximum closure date, assuming that the contract is finished. If the contract is ongoing, the calculation is based on the current date.)",avginstallast24m 3658937A (Average instalments paid by the client over the past 24 months.),avglnamtstart24m 4525187A (Average loan amount in the last 24 months.),avgmaxdpdlast9m 3716943P (Average Days Past Due (DPD) of the client in last 9 months.),avgoutstandbalancel6m 4187114A (Average outstanding balance of applicant for the last 6 months.),avgpmtlast12m 4525200A (Average of payments made by the client in the last 12 months.),bankacctype 710L (Type of applicant's bank account.),cardtype 51L (Type of credit card.),clientscnt12m 3712952L (Number of clients that have used the same mobile phone as the applicant in the past 12 months.),clientscnt3m 3712950L (Number of clients who have the same mobile phone number in the last 3 months.),clientscnt6m 3712949L (Total number of clients who have used the same mobile number in the last 6 months.),clientscnt 100L (Number of applications with matching employer's phone and client's.),clientscnt 1022L (Number of clients sharing the same mobile phone.),clientscnt 1071L (Number of applications where the alternative phone number matches that of the client.),clientscnt 1130L (Number of applications where client's phone number matches the alternative phone contact.),clientscnt 136L (Number of applications associated with same email address as client's email.),clientscnt 157L (Number of clients whose employer has the same phone number as the client.),clientscnt 257L (Number of clients that share an alternative phone number with the applicant.),clientscnt 304L (Number of clients with the same phone number.),clientscnt 360L (Number of clients that have the same alternative phone number and employer's phone number.),clientscnt 493L (Number of clients with matching phone numbers for both the employer and the client.),clientscnt 533L (Number of clients with same client's and alternative's phone number),clientscnt 887L (Number of clients sharing the same employer's phone number.),…,numinstpaidearlyest 4493214L (Number of instalments paid before the due date.),numinstpaidlastcontr 4325080L (Number of paid installments from the client's last contract.),numinstpaidlate1d 3546852L (Number of instalments paid more than 1 day past their due date.),numinstregularpaid 973L (Number of fully paid regular installments in the client's previous contracts.),numinstregularpaidest 4493210L (Number of fully paid regular installments on clients' previous contracts.),numinsttopaygr 769L (Number of unpaid instalments.),numinsttopaygrest 4493213L (Number of unpaid instalments.),numinstunpaidmax 3546851L (Maximum number of unpaid instalments.),numinstunpaidmaxest 4

6495


In [24]:
train_static_2 = preprocess(28)
display(train_static_2.fetch().head())
train_static_2.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,contractssum 5085716L (Total sum of values of contracts retrieved from external credit bureau.),days120 123L (Number of credit bureau queries for the last 120 days.),days180 256L (Number of credit bureau queries for last 180 days.),days30 165L (Number of credit bureau queries for the last 30 days.),days360 512L (Number of Credit Bureau queries for last 360 days.),days90 310L (Number of credit bureau queries for the last 90 days.),description 5085714M (Categorization of clients by credit bureau.),education 1103M (Level of education of the client provided by external source.),education 88M (Education level of the client.),firstquarter 103L (Number of results obtained from credit bureau in the first quarter.),for3years 128L (Number of rejected applications in the past 3 years.),for3years 504L (Client's credit history data over the last three years.),for3years 584L (Number of cancellations in the last 3 years.),formonth 118L (Number of rejections in a month.),formonth 206L (Number of cancelations in the previous month.),formonth 535L (Credit history for the last month.),forquarter 1017L (Number of cancellations recorded in the credit bureau in the last quarter.),forquarter 462L (Number of credit applications that were rejected in the last quarter.),forquarter 634L (Credit history for the last quarter.),fortoday 1092L (Client's credit history for today.),forweek 1077L (Number of cancelations in the last week.),forweek 528L (Credit history for the last week.),forweek 601L (Number of rejected applications in the last week.),foryear 618L (Number of application rejections in the previous year.),foryear 818L (Number of cancelations that occurred in last year.),foryear 850L (Credit history for the last year.),fourthquarter 440L (Number of results in fourth quarter.),maritalst 385M (Marital status of the client.),maritalst 893M (Marital status of the client),numberofqueries 373L (Number of queries to credit bureau.),pmtaverage 3A (Average of tax deductions.),pmtaverage 4527227A (Average of tax deductions.),pmtaverage 4955615A (Average of tax deductions.),pmtcount 4527229L (Number of tax deductions.),pmtcount 4955617L (Number of tax deductions.),pmtcount 693L (Number of tax deductions.),pmtscount 423L (Number of tax deduction payments.),pmtssum 45A (Sum of tax deductions for the client.),requesttype 4525192L (Tax authority request type.),riskassesment 302T (Estimated probability that the client will default on their credit obligation within the next year.),riskassesment 940T (Estimate of client's creditworthiness.),secondquarter 766L (Number of results in second quarter.),thirdquarter 1082L (Number of results in third quarter.)
i32,f32,f32,f32,f32,f32,f32,str,str,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,str,f32,f32,f32
357,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,6.00,6301.40,null,null,null,null,null
381,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,6.00,4019.60,null,null,null,null,null
388,null,6.00,8.00,2.00,10.00,4.00,"""a55475b1""","""a55475b1""","""a55475b1""",2.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6.00,"""a55475b1""","""a55475b1""",10.00,null,null,null,null,null,null,6.00,14548.00,null,null,null,3.00,5.00
405,null,0.00,0.00,0.00,1.00,0.00,"""a55475b1""","""a55475b1""","""a55475b1""",0.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.00,"""a55475b1""","""a55475b1""",1.00,null,null,null,null,null,null,6.00,10498.24,null,null,null,2.00,0.00
409,null,2.00,3.00,0.00,3.00,1.00,"""a55475b1""","""717ddd49"

3618


## Train debitcard

In [25]:
all_files_df.filter(pl.col('filename').str.contains("debit"))

<IPython.core.display.Javascript object>

index,filename
i8,str
21,"""train_debitcard_1.parquet"""


In [26]:
train_debit = preprocess(21)
display(train_debit.fetch().head())
train_debit.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,last180dayaveragebalance 704A (Average balance on debit card in the last 180 days.),last180dayturnover 1134A (Debit card's turnover within the last 180 days.),last30dayturnover 651A (Debit card turnover for the last 30 days.),num_group1
i32,f32,f32,f32,i32
225,null,null,null,0
331,null,null,null,0
358,null,null,null,0
390,null,null,null,0
390,null,null,null,2


3057


In [27]:
all_files_df.filter(pl.col('filename').str.contains("deposit"))

<IPython.core.display.Javascript object>

index,filename
i8,str
22,"""train_deposit_1.parquet"""


In [28]:
train_deposit = preprocess(22)
display(train_deposit.fetch().head())
train_deposit.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,amount 416A (Deposit amount.),num_group1
i32,f32,i32
225,0.00,0
331,260.37,0
358,0.00,0
390,211748.53,0
390,223.68,2


2961


## Train person

In [29]:
all_files_df.filter(pl.col("filename").str.contains("person"))

<IPython.core.display.Javascript object>

index,filename
i8,str
24,"""train_person_1.parquet"""
25,"""train_person_2.parquet"""


In [30]:
train_person_1 = preprocess(24)
display(train_person_1.fetch().head())
train_person_1.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,birth 259D (Date of birth of the person.),childnum 185L (Number of children of the applicant.),contaddr district 15M (Zip code of a contact person's address.),contaddr matchlist 1032L (Indicates whether the contact address is found in a code list.),contaddr smempladdr 334L (Indicates whether the contact address is the same as the employment address.),contaddr zipcode 807M (Zip code of contact address.),education 927M (Education level of the person.),empl employedfrom 271D (Start date of employment.),empl employedtotal 800L (Employment length of a person.),empl industry 691L (Employment Industry of the person.),empladdr district 926M (District where the employer's address is located.),empladdr zipcode 114M (Zipcode of employer's address.),familystate 447L (Family state of the person.),gender 992L (Gender of a person.),housetype 905L (House type of the person.),housingtype 772L (Type of housing of the person.),incometype 1044T (Type of income of the person),isreference 387L (Flag indicating whether the person is a reference contact.),language1 981M (The primary language of the person.),mainoccupationinc 384A (Amount of the main income of the client.),maritalst 703L (Marital status of the client.),num_group1,personindex 1023L (Order of the person specified on the application form.),persontype 1072L (Person type.),persontype 792L (Person type.),registaddr district 1083M (District of person's registered address.),registaddr zipcode 184M (Registered address's zip code of a person.),relationshiptoclient 415T (Relationship to the client.),relationshiptoclient 642T (Relationship to the client.),remitter 829L (Flag indicating whether the client is a remitter.),role 1084L (Type of contact role.),role 993L (Person's role.),safeguarantyflag 411L (Flag indicating if client is using a flexible product with additional safeguard garanty.),sex 738L (Gender of the client.),type 25L (Contact type of a person.)
i32,str,f32,str,bool,bool,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,f32,str,i32,f32,f32,f32,str,str,str,str,bool,str,str,bool,str,str
0,"""1986-07-01""",null,"""P88_18_84""",false,false,"""P167_100_165""","""P97_36_170""","""2017-09-15""","""MORE_FIVE""","""OTHER""","""P142_57_166""","""P167_100_165""","""MARRIED""",null,null,null,"""SALARIED_GOVT""",null,"""P10_39_147""",10800.00,null,0,0.00,1.00,1.00,"""P88_18_84""","""P167_100_165""",null,null,null,"""CL""",null,true,"""F""","""PRIMARY_MOBILE"""
0,null,null,"""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,"""a55475b1""",null,null,1,1.00,1.00,4.00,"""a55475b1""","""a55475b1""","""SPOUSE""",null,false,"""EM""",null,null,null,"""PHONE"""
0,null,null,"""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,"""a55475b1""",null,null,2,2.00,4.00,5.00,"""a55475b1""","""a55475b1""","""COLLEAGUE""","""SPOUSE""",false,"""PE""",null,null,null,"""PHONE"""
0,null,null,"""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,"""a55475b1""",null,null,3,null,5.00,null,"""a55475b1""","""a55475b1""",null,"""COLLEAGUE""",null,"""PE""",null,null,null,"""PHONE"""
1,"""1957-08-01""",null,"""P103_93_94""",false,false,"""P176_37_166""","""P97_36_170""","""2008-10-29""","""MORE_FIVE""","""OTHER""","""P49_46_174""","""P160_59_140""","""DIVORCED""",null,null,null,"""SALARIED_GOVT""",null,"""P10_39_147""",10000.00,null,0,0.00,1.00,1.00,"""P103_93_94""","""P176_37_166""",null,null,null,"""CL""",null,true,"""M""","""PRIMARY_MOBILE"""


4070


In [31]:
train_person_2 = preprocess(25)
display(train_person_2.fetch().head())
train_person_2.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,addres district 368M (District of the person's address.),addres role 871L (Role of person's address.),addres zip 823M (Zip code of the address.),conts role 79M (Type of contact role of a person.),"empls economicalst 849M (The economical status of the person (num_group1 - person, num_group2 - employment).)","empls employedfrom 796D (Start of employment (num_group1 - person, num_group2 - employment).)","empls employer name 740M (Employer's name (num_group1 - person, num_group2 - employment).)",num_group1,num_group2,"relatedpersons role 762T (Relationship type of a client's related person (num_group1 - person, num_group2 - related person).)"
i32,str,str,str,str,str,str,str,i32,i32,str
5,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",0,0,null
6,"""P55_110_32""","""CONTACT""","""P10_68_40""","""P38_92_157""","""P164_110_33""",null,"""a55475b1""",0,0,null
6,"""P55_110_32""","""PERMANENT""","""P10_68_40""","""a55475b1""","""a55475b1""",null,"""a55475b1""",0,1,null
6,"""P204_92_178""","""CONTACT""","""P65_136_169""","""P38_92_157""","""P164_110_33""",null,"""a55475b1""",1,0,"""OTHER_RELATIVE"""
6,"""P191_109_75""","""CONTACT""","""P10_68_40""","""P7_147_157""","""a55475b1""",null,"""a55475b1""",1,1,"""OTHER_RELATIVE"""


2555


In [32]:
all_files_df.filter(pl.col('filename').str.contains("other"))

<IPython.core.display.Javascript object>

index,filename
i8,str
23,"""train_other_1.parquet"""


In [33]:
train_other = preprocess(23)
display(train_other.fetch().head())
train_other.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,amtdebitincoming 4809443A (Incoming debit card transactions amount.),amtdebitoutgoing 4809440A (Outgoing debit card transactions amount.),amtdepositbalance 4809441A (Deposit balance of client.),amtdepositincoming 4809444A (Amount of incoming deposits to client's account.),amtdepositoutgoing 4809442A (Amount of outgoing deposits from client's account.),num_group1
i32,f32,f32,f32,f32,f32,i32
43801,12466.60,12291.20,914.20,0.00,304.80,0
43991,3333.40,3273.40,0.00,0.00,0.00,0
44001,10000.00,10000.00,0.00,0.00,0.00,0
44053,0.00,0.00,2586.40,0.00,88.80,0
44130,63.80,60.80,0.00,0.00,0.00,0


3096


## Train Tax Registry

In [34]:
all_files_df.filter(pl.col("filename").str.contains("registry"))

<IPython.core.display.Javascript object>

index,filename
i8,str
29,"""train_tax_registry_a_1.parquet"""
30,"""train_tax_registry_b_1.parquet"""
31,"""train_tax_registry_c_1.parquet"""


In [35]:
train_tax_registry_1 = preprocess(29)
display(train_tax_registry_1.fetch().head())
train_tax_registry_1.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,amount 4527230A (Tax deductions amount tracked by the government registry.),name 4527232M (Name of employer.),num_group1
i32,f32,str,i32
28631,1946.00,"""f980a1ea""",2
28631,711.00,"""f980a1ea""",3
28631,3616.40,"""f980a1ea""",0
28631,2600.00,"""f980a1ea""",1
28632,2564.00,"""5f9b74f5""",5


2993


In [36]:
train_tax_registry_2 = preprocess(30)
display(train_tax_registry_2.fetch().head())
train_tax_registry_2.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,amount 4917619A (Tax deductions amount tracked by the government registry.),name 4917606M (Name of employer.),num_group1
i32,f32,str,i32
49435,6885.00,"""6b730375""",7
49435,6885.00,"""6b730375""",1
49435,6885.00,"""6b730375""",8
49435,6885.00,"""6b730375""",3
49435,6885.00,"""6b730375""",4


2993


In [37]:
train_tax_registry_3 = preprocess(31)
display(train_tax_registry_3.fetch().head())
train_tax_registry_3.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,employername 160M (Employer's name.),num_group1,pmtamount 36A (Tax deductions amount for credit bureau payments.)
i32,str,i32,f32
357,"""c91b12ff""",5,1100.00
357,"""c91b12ff""",1,1200.00
357,"""c91b12ff""",4,1200.00
357,"""c91b12ff""",0,1200.00
357,"""c91b12ff""",3,628.60


3057


## Train credit Bureau

In [38]:
train_credit_1 = preprocess(4)
display(train_credit_1.fetch().head(2))
gc.collect()
train_credit_1.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,annualeffectiverate 199L (Interest rate of the closed contracts.),annualeffectiverate 63L (Interest rate for the active contracts.),classificationofcontr 13M (Classificiation of the active contract.),classificationofcontr 400M (Classificiation of the closed contract.),contractst 545M (Contract status.),contractst 964M (Contract status of terminated credit contract.),contractsum 5085717L (Sum of other contract values.),credlmt 230A (Credit limit of the closed credit contracts from credit bureau.),credlmt 935A (Credit limit for active loan.),debtoutstand 525A (Outstanding amount of existing contract.),debtoverdue 47A (Amount that is currently past due on a client's existing credit contract.),description 351M (Categorization of clients by credit bureau.),dpdmax 139P (Maximal days past due for active contract.),dpdmax 757P (Maximum days past due for a closed contract.),financialinstitution 382M (Name of financial institution that is linked to a closed contract.),financialinstitution 591M (Financial institution name of the active contract.),instlamount 768A (Instalment amount for the active contract in credit bureau.),instlamount 852A (Instalment amount for closed contract.),interestrate 508L (Interest rate for a closed contract in the credit bureau.),monthlyinstlamount 332A (Monthly instalment amount for active contract.),monthlyinstlamount 674A (Monthly amount of instalment payment on a closed contract.),nominalrate 281L (Interest rate of the active contract.),nominalrate 498L (Interest rate for closed contract.),num_group1,numberofcontrsvalue 258L (Number of active contracts in credit bureau.),numberofcontrsvalue 358L (Number of closed credit contracts.),numberofinstls 229L (Number of instalments on closed contract.),numberofinstls 320L (Number of instalments of the active contract.),numberofoutstandinstls 520L (Number of outstanding instalment for closed contract.),numberofoutstandinstls 59L (Number of outstanding instalments for the active contracts.),numberofoverdueinstlmax 1039L (Number of outstanding instalments for active contracts.),numberofoverdueinstlmax 1151L (Maximum number of past due installments for a closed contract.),numberofoverdueinstlmaxdat 148D (Date of maximum number of past due instalments for the closed contract.),numberofoverdueinstlmaxdat 641D (Date of maximum number of past due instalments for the active contract.),numberofoverdueinstls 725L (Maximum number of past due instalments for an active contract.),numberofoverdueinstls 834L (Number of past due instalments for a closed contract.),outstandingamount 354A (Outstanding amount for closed credit contract in credit bureau.),outstandingamount 362A (Active contract's outstanding amount.),overdueamount 31A (Past due amount for a closed contract.),overdueamount 659A (Past due amount for active contract.),overdueamountmax2 14A (Maximal past due amount for an active contract.),overdueamountmax2 398A (Maximal overdue amount for a closed contract.),overdueamountmax 155A (Maximal past due amount for active contract.),overdueamountmax 35A (Maximal past due amount for a closed contract.),periodicityofpmts 1102L (Frequency of instalments for a closed contract.),periodicityofpmts 837L (Frequency of instalments for an active contract.),prolongationcount 1120L (Count of prolongations on terminated contract according to credit bureau.),prolongationcount 599L (Count of active contract prolongations.),purposeofcred 426M (Purpose of credit for active contract.),purposeofcred 874M (Purpose of credit on a closed contract.),residualamount 488A (Residual amount of a closed contract.),residualamount 856A (Residual amount for the active contract.),subjectrole 182M (Subject role in active credit contract.),subjectrole 93M (Subject role in closed credit contract.),totalamount 6A (Total amount of closed contracts.),totalamount 996A (Total amount of active contracts in the credit bureau.),totaldebtoverduevalue 178A (Total amount of past due debt on active contracts.),totald

4078


In [39]:
train_credit_2 = preprocess(5)
display(train_credit_2.fetch().head(2))
train_credit_2.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,annualeffectiverate 199L (Interest rate of the closed contracts.),annualeffectiverate 63L (Interest rate for the active contracts.),classificationofcontr 13M (Classificiation of the active contract.),classificationofcontr 400M (Classificiation of the closed contract.),contractst 545M (Contract status.),contractst 964M (Contract status of terminated credit contract.),contractsum 5085717L (Sum of other contract values.),credlmt 230A (Credit limit of the closed credit contracts from credit bureau.),credlmt 935A (Credit limit for active loan.),debtoutstand 525A (Outstanding amount of existing contract.),debtoverdue 47A (Amount that is currently past due on a client's existing credit contract.),description 351M (Categorization of clients by credit bureau.),dpdmax 139P (Maximal days past due for active contract.),dpdmax 757P (Maximum days past due for a closed contract.),financialinstitution 382M (Name of financial institution that is linked to a closed contract.),financialinstitution 591M (Financial institution name of the active contract.),instlamount 768A (Instalment amount for the active contract in credit bureau.),instlamount 852A (Instalment amount for closed contract.),interestrate 508L (Interest rate for a closed contract in the credit bureau.),monthlyinstlamount 332A (Monthly instalment amount for active contract.),monthlyinstlamount 674A (Monthly amount of instalment payment on a closed contract.),nominalrate 281L (Interest rate of the active contract.),nominalrate 498L (Interest rate for closed contract.),num_group1,numberofcontrsvalue 258L (Number of active contracts in credit bureau.),numberofcontrsvalue 358L (Number of closed credit contracts.),numberofinstls 229L (Number of instalments on closed contract.),numberofinstls 320L (Number of instalments of the active contract.),numberofoutstandinstls 520L (Number of outstanding instalment for closed contract.),numberofoutstandinstls 59L (Number of outstanding instalments for the active contracts.),numberofoverdueinstlmax 1039L (Number of outstanding instalments for active contracts.),numberofoverdueinstlmax 1151L (Maximum number of past due installments for a closed contract.),numberofoverdueinstlmaxdat 148D (Date of maximum number of past due instalments for the closed contract.),numberofoverdueinstlmaxdat 641D (Date of maximum number of past due instalments for the active contract.),numberofoverdueinstls 725L (Maximum number of past due instalments for an active contract.),numberofoverdueinstls 834L (Number of past due instalments for a closed contract.),outstandingamount 354A (Outstanding amount for closed credit contract in credit bureau.),outstandingamount 362A (Active contract's outstanding amount.),overdueamount 31A (Past due amount for a closed contract.),overdueamount 659A (Past due amount for active contract.),overdueamountmax2 14A (Maximal past due amount for an active contract.),overdueamountmax2 398A (Maximal overdue amount for a closed contract.),overdueamountmax 155A (Maximal past due amount for active contract.),overdueamountmax 35A (Maximal past due amount for a closed contract.),periodicityofpmts 1102L (Frequency of instalments for a closed contract.),periodicityofpmts 837L (Frequency of instalments for an active contract.),prolongationcount 1120L (Count of prolongations on terminated contract according to credit bureau.),prolongationcount 599L (Count of active contract prolongations.),purposeofcred 426M (Purpose of credit for active contract.),purposeofcred 874M (Purpose of credit on a closed contract.),residualamount 488A (Residual amount of a closed contract.),residualamount 856A (Residual amount for the active contract.),subjectrole 182M (Subject role in active credit contract.),subjectrole 93M (Subject role in closed credit contract.),totalamount 6A (Total amount of closed contracts.),totalamount 996A (Total amount of active contracts in the credit bureau.),totaldebtoverduevalue 178A (Total amount of past due debt on active contracts.),totald

59


In [40]:
train_credit_3 = preprocess(6)
display(train_credit_3.fetch().head(2))
train_credit_3.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,annualeffectiverate 199L (Interest rate of the closed contracts.),annualeffectiverate 63L (Interest rate for the active contracts.),classificationofcontr 13M (Classificiation of the active contract.),classificationofcontr 400M (Classificiation of the closed contract.),contractst 545M (Contract status.),contractst 964M (Contract status of terminated credit contract.),contractsum 5085717L (Sum of other contract values.),credlmt 230A (Credit limit of the closed credit contracts from credit bureau.),credlmt 935A (Credit limit for active loan.),debtoutstand 525A (Outstanding amount of existing contract.),debtoverdue 47A (Amount that is currently past due on a client's existing credit contract.),description 351M (Categorization of clients by credit bureau.),dpdmax 139P (Maximal days past due for active contract.),dpdmax 757P (Maximum days past due for a closed contract.),financialinstitution 382M (Name of financial institution that is linked to a closed contract.),financialinstitution 591M (Financial institution name of the active contract.),instlamount 768A (Instalment amount for the active contract in credit bureau.),instlamount 852A (Instalment amount for closed contract.),interestrate 508L (Interest rate for a closed contract in the credit bureau.),monthlyinstlamount 332A (Monthly instalment amount for active contract.),monthlyinstlamount 674A (Monthly amount of instalment payment on a closed contract.),nominalrate 281L (Interest rate of the active contract.),nominalrate 498L (Interest rate for closed contract.),num_group1,numberofcontrsvalue 258L (Number of active contracts in credit bureau.),numberofcontrsvalue 358L (Number of closed credit contracts.),numberofinstls 229L (Number of instalments on closed contract.),numberofinstls 320L (Number of instalments of the active contract.),numberofoutstandinstls 520L (Number of outstanding instalment for closed contract.),numberofoutstandinstls 59L (Number of outstanding instalments for the active contracts.),numberofoverdueinstlmax 1039L (Number of outstanding instalments for active contracts.),numberofoverdueinstlmax 1151L (Maximum number of past due installments for a closed contract.),numberofoverdueinstlmaxdat 148D (Date of maximum number of past due instalments for the closed contract.),numberofoverdueinstlmaxdat 641D (Date of maximum number of past due instalments for the active contract.),numberofoverdueinstls 725L (Maximum number of past due instalments for an active contract.),numberofoverdueinstls 834L (Number of past due instalments for a closed contract.),outstandingamount 354A (Outstanding amount for closed credit contract in credit bureau.),outstandingamount 362A (Active contract's outstanding amount.),overdueamount 31A (Past due amount for a closed contract.),overdueamount 659A (Past due amount for active contract.),overdueamountmax2 14A (Maximal past due amount for an active contract.),overdueamountmax2 398A (Maximal overdue amount for a closed contract.),overdueamountmax 155A (Maximal past due amount for active contract.),overdueamountmax 35A (Maximal past due amount for a closed contract.),periodicityofpmts 1102L (Frequency of instalments for a closed contract.),periodicityofpmts 837L (Frequency of instalments for an active contract.),prolongationcount 1120L (Count of prolongations on terminated contract according to credit bureau.),prolongationcount 599L (Count of active contract prolongations.),purposeofcred 426M (Purpose of credit for active contract.),purposeofcred 874M (Purpose of credit on a closed contract.),residualamount 488A (Residual amount of a closed contract.),residualamount 856A (Residual amount for the active contract.),subjectrole 182M (Subject role in active credit contract.),subjectrole 93M (Subject role in closed credit contract.),totalamount 6A (Total amount of closed contracts.),totalamount 996A (Total amount of active contracts in the credit bureau.),totaldebtoverduevalue 178A (Total amount of past due debt on active contracts.),totald

4597


In [41]:
train_credit_4 = preprocess(7)
display(train_credit_4.fetch().head(2))
train_credit_4.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,annualeffectiverate 199L (Interest rate of the closed contracts.),annualeffectiverate 63L (Interest rate for the active contracts.),classificationofcontr 13M (Classificiation of the active contract.),classificationofcontr 400M (Classificiation of the closed contract.),contractst 545M (Contract status.),contractst 964M (Contract status of terminated credit contract.),contractsum 5085717L (Sum of other contract values.),credlmt 230A (Credit limit of the closed credit contracts from credit bureau.),credlmt 935A (Credit limit for active loan.),debtoutstand 525A (Outstanding amount of existing contract.),debtoverdue 47A (Amount that is currently past due on a client's existing credit contract.),description 351M (Categorization of clients by credit bureau.),dpdmax 139P (Maximal days past due for active contract.),dpdmax 757P (Maximum days past due for a closed contract.),financialinstitution 382M (Name of financial institution that is linked to a closed contract.),financialinstitution 591M (Financial institution name of the active contract.),instlamount 768A (Instalment amount for the active contract in credit bureau.),instlamount 852A (Instalment amount for closed contract.),interestrate 508L (Interest rate for a closed contract in the credit bureau.),monthlyinstlamount 332A (Monthly instalment amount for active contract.),monthlyinstlamount 674A (Monthly amount of instalment payment on a closed contract.),nominalrate 281L (Interest rate of the active contract.),nominalrate 498L (Interest rate for closed contract.),num_group1,numberofcontrsvalue 258L (Number of active contracts in credit bureau.),numberofcontrsvalue 358L (Number of closed credit contracts.),numberofinstls 229L (Number of instalments on closed contract.),numberofinstls 320L (Number of instalments of the active contract.),numberofoutstandinstls 520L (Number of outstanding instalment for closed contract.),numberofoutstandinstls 59L (Number of outstanding instalments for the active contracts.),numberofoverdueinstlmax 1039L (Number of outstanding instalments for active contracts.),numberofoverdueinstlmax 1151L (Maximum number of past due installments for a closed contract.),numberofoverdueinstlmaxdat 148D (Date of maximum number of past due instalments for the closed contract.),numberofoverdueinstlmaxdat 641D (Date of maximum number of past due instalments for the active contract.),numberofoverdueinstls 725L (Maximum number of past due instalments for an active contract.),numberofoverdueinstls 834L (Number of past due instalments for a closed contract.),outstandingamount 354A (Outstanding amount for closed credit contract in credit bureau.),outstandingamount 362A (Active contract's outstanding amount.),overdueamount 31A (Past due amount for a closed contract.),overdueamount 659A (Past due amount for active contract.),overdueamountmax2 14A (Maximal past due amount for an active contract.),overdueamountmax2 398A (Maximal overdue amount for a closed contract.),overdueamountmax 155A (Maximal past due amount for active contract.),overdueamountmax 35A (Maximal past due amount for a closed contract.),periodicityofpmts 1102L (Frequency of instalments for a closed contract.),periodicityofpmts 837L (Frequency of instalments for an active contract.),prolongationcount 1120L (Count of prolongations on terminated contract according to credit bureau.),prolongationcount 599L (Count of active contract prolongations.),purposeofcred 426M (Purpose of credit for active contract.),purposeofcred 874M (Purpose of credit on a closed contract.),residualamount 488A (Residual amount of a closed contract.),residualamount 856A (Residual amount for the active contract.),subjectrole 182M (Subject role in active credit contract.),subjectrole 93M (Subject role in closed credit contract.),totalamount 6A (Total amount of closed contracts.),totalamount 996A (Total amount of active contracts in the credit bureau.),totaldebtoverduevalue 178A (Total amount of past due debt on active contracts.),totald

7266


In [42]:
(
    all_files_df
    .filter(pl.col("filename").str.contains("credit"))
)

<IPython.core.display.Javascript object>

index,filename
i8,str
4,"""train_credit_bureau_a_1_0.parquet"""
5,"""train_credit_bureau_a_1_1.parquet"""
6,"""train_credit_bureau_a_1_2.parquet"""
7,"""train_credit_bureau_a_1_3.parquet"""
8,"""train_credit_bureau_a_2_0.parquet"""
9,"""train_credit_bureau_a_2_1.parquet"""
10,"""train_credit_bureau_a_2_10.parquet"""
11,"""train_credit_bureau_a_2_2.parquet"""
12,"""train_credit_bureau_a_2_3.parquet"""


In [43]:
train_credit_5 = preprocess(8)
display(train_credit_5.fetch().head(2))
train_credit_5.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,collater typofvalofguarant 298M (Collateral valuation type (active contract).),collater typofvalofguarant 407M (Collateral valuation type (closed contract).),collater valueofguarantee 1124L (Value of collateral for active contract.),collater valueofguarantee 876L (Value of collateral for closed contract.),collaterals typeofguarante 359M (Type of collateral that was used as a guarantee for a closed contract.),collaterals typeofguarante 669M (Collateral type for the active contract.),num_group1,num_group2,"pmts dpd 1073P (Days past due of the payment for the active contract (num_group1 - existing contract, num_group2 - payment).)","pmts dpd 303P (Days past due of the payment for terminated contract according to credit bureau (num_group1 - terminated contract, num_group2 - payment).)","pmts month 158T (Month of payment for a closed contract (num_group1 - existing contract, num_group2 - payment).)","pmts month 706T (Month of payment for active contract (num_group1 - terminated contract, num_group2 - payment).)","pmts overdue 1140A (Overdue payment for an active contract (num_group1 - existing contract, num_group2 - payment).)","pmts overdue 1152A (Overdue payment for a closed contract (num_group1 - terminated contract, num_group2 - payment).)","pmts year 1139T (Year of payment for an active contract (num_group1 - existing contract, num_group2 - payment).)","pmts year 507T (Payment year for a closed credit contract (num_group1 - terminated contract, num_group2 - payment).)","subjectroles name 541M (Name of subject role in closed credit contract (num_group1 - terminated contract, num_group2 - subject roles).)","subjectroles name 838M (Name of subject role in active credit contract (num_group1 - existing contract, num_group2 - subject roles).)"
i32,str,str,f32,f32,str,str,i32,i32,f32,f32,f32,f32,f32,f32,f32,f32,str,str
388,"""8fd95e4b""","""a55475b1""",0.00,null,"""a55475b1""","""c7a5ad39""",0,0,null,null,2.00,null,null,null,2018.00,null,"""a55475b1""","""ab3c25cf"""
388,"""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",0,1,null,null,3.00,null,null,null,2018.00,null,"""a55475b1""","""a55475b1"""


2118


In [44]:
train_credit_6 = preprocess(9)
display(train_credit_6.fetch().head(2))
train_credit_6.pipe(plot_nulls)
print(gc.collect())

<IPython.core.display.Javascript object>

case_id,collater typofvalofguarant 298M (Collateral valuation type (active contract).),collater typofvalofguarant 407M (Collateral valuation type (closed contract).),collater valueofguarantee 1124L (Value of collateral for active contract.),collater valueofguarantee 876L (Value of collateral for closed contract.),collaterals typeofguarante 359M (Type of collateral that was used as a guarantee for a closed contract.),collaterals typeofguarante 669M (Collateral type for the active contract.),num_group1,num_group2,"pmts dpd 1073P (Days past due of the payment for the active contract (num_group1 - existing contract, num_group2 - payment).)","pmts dpd 303P (Days past due of the payment for terminated contract according to credit bureau (num_group1 - terminated contract, num_group2 - payment).)","pmts month 158T (Month of payment for a closed contract (num_group1 - existing contract, num_group2 - payment).)","pmts month 706T (Month of payment for active contract (num_group1 - terminated contract, num_group2 - payment).)","pmts overdue 1140A (Overdue payment for an active contract (num_group1 - existing contract, num_group2 - payment).)","pmts overdue 1152A (Overdue payment for a closed contract (num_group1 - terminated contract, num_group2 - payment).)","pmts year 1139T (Year of payment for an active contract (num_group1 - existing contract, num_group2 - payment).)","pmts year 507T (Payment year for a closed credit contract (num_group1 - terminated contract, num_group2 - payment).)","subjectroles name 541M (Name of subject role in closed credit contract (num_group1 - terminated contract, num_group2 - subject roles).)","subjectroles name 838M (Name of subject role in active credit contract (num_group1 - existing contract, num_group2 - subject roles).)"
i32,str,str,f32,f32,str,str,i32,i32,f32,f32,f32,f32,f32,f32,f32,f32,str,str
6683,"""8fd95e4b""","""a55475b1""",6742800.00,null,"""a55475b1""","""9276e4bb""",0,0,null,null,2.00,null,null,null,2017.00,null,"""a55475b1""","""ab3c25cf"""
6683,"""8fd95e4b""","""a55475b1""",6700000.00,null,"""a55475b1""","""0e63c0f0""",0,1,null,null,3.00,null,null,null,2017.00,null,"""a55475b1""","""a55475b1"""


2836


- All the credit bureau from "a" internal source have the same columns but they are divided by the depth of their group.
- All the credit bureau from "b" too have the same columns.
- After clubbing the dataframes with same columns we get

In [45]:
train_credit_internal = (
    pl.concat(
        [preprocess(_).with_columns(pl.col(pl.INTEGER_DTYPES).cast(pl.Int32)) for _ in range(4,8)],
        how="vertical"
    )
)
train_credit_internal.fetch().head()

<IPython.core.display.Javascript object>

case_id,annualeffectiverate 199L (Interest rate of the closed contracts.),annualeffectiverate 63L (Interest rate for the active contracts.),classificationofcontr 13M (Classificiation of the active contract.),classificationofcontr 400M (Classificiation of the closed contract.),contractst 545M (Contract status.),contractst 964M (Contract status of terminated credit contract.),contractsum 5085717L (Sum of other contract values.),credlmt 230A (Credit limit of the closed credit contracts from credit bureau.),credlmt 935A (Credit limit for active loan.),debtoutstand 525A (Outstanding amount of existing contract.),debtoverdue 47A (Amount that is currently past due on a client's existing credit contract.),description 351M (Categorization of clients by credit bureau.),dpdmax 139P (Maximal days past due for active contract.),dpdmax 757P (Maximum days past due for a closed contract.),financialinstitution 382M (Name of financial institution that is linked to a closed contract.),financialinstitution 591M (Financial institution name of the active contract.),instlamount 768A (Instalment amount for the active contract in credit bureau.),instlamount 852A (Instalment amount for closed contract.),interestrate 508L (Interest rate for a closed contract in the credit bureau.),monthlyinstlamount 332A (Monthly instalment amount for active contract.),monthlyinstlamount 674A (Monthly amount of instalment payment on a closed contract.),nominalrate 281L (Interest rate of the active contract.),nominalrate 498L (Interest rate for closed contract.),num_group1,numberofcontrsvalue 258L (Number of active contracts in credit bureau.),numberofcontrsvalue 358L (Number of closed credit contracts.),numberofinstls 229L (Number of instalments on closed contract.),numberofinstls 320L (Number of instalments of the active contract.),numberofoutstandinstls 520L (Number of outstanding instalment for closed contract.),numberofoutstandinstls 59L (Number of outstanding instalments for the active contracts.),numberofoverdueinstlmax 1039L (Number of outstanding instalments for active contracts.),numberofoverdueinstlmax 1151L (Maximum number of past due installments for a closed contract.),numberofoverdueinstlmaxdat 148D (Date of maximum number of past due instalments for the closed contract.),numberofoverdueinstlmaxdat 641D (Date of maximum number of past due instalments for the active contract.),numberofoverdueinstls 725L (Maximum number of past due instalments for an active contract.),numberofoverdueinstls 834L (Number of past due instalments for a closed contract.),outstandingamount 354A (Outstanding amount for closed credit contract in credit bureau.),outstandingamount 362A (Active contract's outstanding amount.),overdueamount 31A (Past due amount for a closed contract.),overdueamount 659A (Past due amount for active contract.),overdueamountmax2 14A (Maximal past due amount for an active contract.),overdueamountmax2 398A (Maximal overdue amount for a closed contract.),overdueamountmax 155A (Maximal past due amount for active contract.),overdueamountmax 35A (Maximal past due amount for a closed contract.),periodicityofpmts 1102L (Frequency of instalments for a closed contract.),periodicityofpmts 837L (Frequency of instalments for an active contract.),prolongationcount 1120L (Count of prolongations on terminated contract according to credit bureau.),prolongationcount 599L (Count of active contract prolongations.),purposeofcred 426M (Purpose of credit for active contract.),purposeofcred 874M (Purpose of credit on a closed contract.),residualamount 488A (Residual amount of a closed contract.),residualamount 856A (Residual amount for the active contract.),subjectrole 182M (Subject role in active credit contract.),subjectrole 93M (Subject role in closed credit contract.),totalamount 6A (Total amount of closed contracts.),totalamount 996A (Total amount of active contracts in the credit bureau.),totaldebtoverduevalue 178A (Total amount of past due debt on active contracts.),totald

In [46]:
train_credit_internal_depth = (
    pl.concat(
        [preprocess(_).with_columns(cs.integer().cast(pl.Int32)) for _ in range(8,19)],
        how="vertical"
    )
)
train_credit_internal_depth.fetch().head()

<IPython.core.display.Javascript object>

case_id,collater typofvalofguarant 298M (Collateral valuation type (active contract).),collater typofvalofguarant 407M (Collateral valuation type (closed contract).),collater valueofguarantee 1124L (Value of collateral for active contract.),collater valueofguarantee 876L (Value of collateral for closed contract.),collaterals typeofguarante 359M (Type of collateral that was used as a guarantee for a closed contract.),collaterals typeofguarante 669M (Collateral type for the active contract.),num_group1,num_group2,"pmts dpd 1073P (Days past due of the payment for the active contract (num_group1 - existing contract, num_group2 - payment).)","pmts dpd 303P (Days past due of the payment for terminated contract according to credit bureau (num_group1 - terminated contract, num_group2 - payment).)","pmts month 158T (Month of payment for a closed contract (num_group1 - existing contract, num_group2 - payment).)","pmts month 706T (Month of payment for active contract (num_group1 - terminated contract, num_group2 - payment).)","pmts overdue 1140A (Overdue payment for an active contract (num_group1 - existing contract, num_group2 - payment).)","pmts overdue 1152A (Overdue payment for a closed contract (num_group1 - terminated contract, num_group2 - payment).)","pmts year 1139T (Year of payment for an active contract (num_group1 - existing contract, num_group2 - payment).)","pmts year 507T (Payment year for a closed credit contract (num_group1 - terminated contract, num_group2 - payment).)","subjectroles name 541M (Name of subject role in closed credit contract (num_group1 - terminated contract, num_group2 - subject roles).)","subjectroles name 838M (Name of subject role in active credit contract (num_group1 - existing contract, num_group2 - subject roles).)"
i32,str,str,f32,f32,str,str,i32,i32,f32,f32,f32,f32,f32,f32,f32,f32,str,str
388,"""8fd95e4b""","""a55475b1""",0.00,null,"""a55475b1""","""c7a5ad39""",0,0,null,null,2.00,null,null,null,2018.00,null,"""a55475b1""","""ab3c25cf"""
388,"""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",0,1,null,null,3.00,null,null,null,2018.00,null,"""a55475b1""","""a55475b1"""
388,"""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",0,2,null,null,4.00,null,null,null,2018.00,null,"""a55475b1""","""a55475b1"""
388,"""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",0,3,null,null,5.00,null,null,null,2018.00,null,"""a55475b1""","""a55475b1"""
388,"""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""",0,4,null,null,6.00,null,null,null,2018.00,null,"""a55475b1""","""a55475b1"""


In [47]:
# working with more variables will be difficult in colab
del train_credit_1,train_credit_2,train_credit_3,train_credit_4,train_credit_5,train_credit_6

<IPython.core.display.Javascript object>

# Combining and Analysing

In [48]:
def plot_corr(df):
    return (
        df
        .pipe(drop_null_columns)
        .corr()
        .select(pl.all().round(2))
        .pipe(
            lambda df:
            px.imshow(df,color_continuous_scale="twilight",text_auto=True)
            .update_yaxes(ticktext=df.columns,tickvals=list(range(len(df.columns))))
            .show()
        )
    )

<IPython.core.display.Javascript object>

In [49]:
train_base = (
    train_base
    .drop("MONTH","WEEK_NUM")
    .with_columns(
        pl.col("date_decision").dt.month().alias("date_month"),
        pl.col("date_decision").dt.day().alias("date_day"),
        pl.col("date_decision").dt.year().alias("date_year"),
        pl.col("date_decision").dt.week().alias("date_week")
    )
)
(
    train_base
    .pipe(plot_corr)
)

<IPython.core.display.Javascript object>

In [50]:
# (
#     train_previous_application
#     .join(
#         train_base.select(["case_id","target"]).lazy(),
#         left_on="case_id",
#         right_on="case_id"
#     )
#     .pipe(plot_corr)
# )

<IPython.core.display.Javascript object>

In [58]:
import psutil

print(psutil.cpu_count())
num_cpus = psutil.cpu_count()
cpu_speeds = []
psutil.cpu_freq(percpu=True)

<IPython.core.display.Javascript object>

8


[scpufreq(current=2199.998, min=0.0, max=0.0),
 scpufreq(current=2199.998, min=0.0, max=0.0),
 scpufreq(current=2199.998, min=0.0, max=0.0),
 scpufreq(current=2199.998, min=0.0, max=0.0),
 scpufreq(current=2199.998, min=0.0, max=0.0),
 scpufreq(current=2199.998, min=0.0, max=0.0),
 scpufreq(current=2199.998, min=0.0, max=0.0),
 scpufreq(current=2199.998, min=0.0, max=0.0)]